In [69]:
def pwr(x: int, power: int, modulo: int) -> int:
    '''
        calc x ^ power % modulo
    '''
    if power == 0:
        return 1
    if power % 2 == 1:
        res = pwr(x, power - 1, modulo) * x
        return res % modulo
    res = pwr(x, power // 2, modulo)
    res *= res
    return res % modulo


def gcd_ext(x: int, y: int):
    '''
        calc greatest common divisor, also return a, b such that x * a + b * y = gcd(x,y)
    '''
    if x == 0:
        return y, 0, 1
    d, a1, b1 = gcd_ext(y % x, x)
    a = b1 - (y // x) * a1
    b = a1
    return d, a, b

def inverse(x: int, modulo: int):
    '''
        find y such that x * y % modulo = 1
        make sure that gcd(x, modulo) = 1
    '''
    d, a, b = gcd_ext(x, modulo)
    assert d == 1, d
    a %= modulo
    assert 0 <= a < modulo, a
    return a

In [48]:
gcd_ext(12, 18)

(6, -1, 1)

In [49]:
pwr(2, 3, 12)

8

In [50]:
assert (inverse(7, 114) * 7) % 114 == 1

In [109]:
MODULO = 340282366920938463463374607431768211297  # <- is prime, 2 ** 128 - 159

In [110]:
cipher(3), cipher(4)

(262025822871216288459711354036727624962, 6627262464)

In [111]:
decipher(cipher(3)), decipher(cipher(4))

(3, 4)

In [112]:
class PowerCipher:
    '''
        Cipher is based on property that it's too hard to find power in equasion: (a ^ power) % modulo = b
        when a,b,modulo are known
        so power is kind of private key
    '''
    def __init__(self, power: int, prime_modulo: int):
        self._power = power
        self._prime_modulo = prime_modulo  # here we won't check if modulo is prime, make it sure yourself
        # it's necessary property to find inverse power that gcd(power, prime_modulo - 1) = 1, it will be checked in next line
        self._inv_power = inverse(power, prime_modulo - 1)  # since Euler_function(prime_modulo) = prime_modulo - 1
        
    def cipher(self, msg: int):
        assert 1 < msg < self._prime_modulo
        return pwr(msg, self._power, self._prime_modulo)
    
    def decipher(self, msg: int):
        assert 1 < msg < self._prime_modulo
        return pwr(msg, self._inv_power, self._prime_modulo)

In [113]:
private1 = 1003
private2 = 725

c1 = PowerCipher(private1, MODULO)
c2 = PowerCipher(private2, MODULO)

for i in range(2, 2 + 52):
    assert i == c1.decipher(c1.cipher(i))
    assert i == c2.decipher(c2.cipher(i))
    assert c1.cipher(c2.cipher(i)) == c2.cipher(c1.cipher(i))

In [114]:
13159035226703211890654003961158071376254143192955391095350192585677108936704 < 2 ** 255

True

In [115]:
for i in range(52):
    p1 = c1.cipher(i + 2)
    p21 = c2.cipher(p1)
    pm121 = c1.decipher(p21)
    print(i, p1, p21, pm121)

0 205381139980068277633160390919071830822 212280790414654054732803390782333444079 3931291551347297504256290361566035968
1 102556355728778912194778419807133212971 304186157023269436326735785247893556677 127826784364580954733204442146997678188
2 132368017470289613671544999873657844602 252346015100414562768783879476735644610 7221512800809898655060050269947559936
3 9573848775524422762089489004757727986 228893512955715537785861189498162292363 263690366206133228884600028611432463532
4 231970353719609935446390320577287225138 246786824519780353272677001831108785965 123847454697520124946205291478140896941
5 98602134968864167172559311264648553796 63988760678677982529534721990068509352 113777784632330336208448407682070470530
6 117027839253373852167987362152100934558 88562481658940908197530029595645230048 183406606910019483252109870047643006208
7 335771217084156640134054775466344226919 312482675849707978578507458811078468008 103908358681394420460512259012133656714
8 6663478131357650783424503656107